# Loading the reference images

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
references_dir = data_dir / "cloth_competition_references_0002"
teams_dir = data_dir / "evaluation_icra_2024"
grasp_success_csv = data_dir / "evaluation_icra_2024_grasp_success.csv"

# team_name = "AIR_JNU"
# team_name = "Team_Ljubljana"
# team_name = "Ewha_Glab"
team_name = "SCUT-ROBOT"
# team_name = "test_greater_bay"
# team_name = "SRCB-Test"
# team_name = "Shibata_Lab"
# team_name = "ai_robot"
# team_name = "UOS_robotics"
# team_name = "AIS_team"
# team_name = "3c1s"
team_dir = teams_dir / team_name

team_dir

In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from cloth_tools.dataset.format import load_competition_observation

data_dir = Path("data")
# references_dir = data_dir / "cloth_competition_references_0001"
references_dir = data_dir / "cloth_competition_references_0002"
teams_dir = data_dir / "evaluation_icra_2024"
grasp_success_csv = data_dir / "evaluation_icra_2024_grasp_success.csv"

# team_name = "AIR_JNU"
# team_name = "Team_Ljubljana"
# team_name = "Ewha_Glab"
team_name = "SCUT-ROBOT"
# team_name = "test_greater_bay"
# team_name = "SRCB-Test"
# team_name = "Shibata_Lab"
# team_name = "ai_robot"
# team_name = "UOS_robotics"
# team_name = "AIS_team"
# team_name = "3c1s"
team_dir = teams_dir / team_name

team_dir

In [ ]:
os.path.exists(references_dir)

In [ ]:
reference_sample_dirs = [references_dir / dirname for dirname in sorted(os.listdir(references_dir))]
reference_sample_dirs

In [ ]:
reference_observation_dirs = [sample_dir / "observation_result" for sample_dir in reference_sample_dirs]
# reference_observation_dirs = reference_sample_dirs # FOR THE OLD REFERENCE FOLDER STRUCTURE

# observation_dirs = [dataset_dir / ref_dir for ref_dir in sorted(os.listdir(dataset_dir))]
len(reference_observation_dirs)

In [ ]:
import json

reference_images = {}
reference_surface_areas = {}

for cloth_id, observation_dir in enumerate(reference_observation_dirs):
    observation = load_competition_observation(observation_dir)

    results_file = observation_dir / "result.json"

    if results_file.exists():
        with open(results_file, "r") as f:
            result = json.load(f)
        surface_area = result["coverage"]
        # reference_images[f"Evaluation reference {cloth_id} ({surface_area:.3f} m²)"] = observation.image_left
        reference_images[f"Evaluation reference {cloth_id}"] = observation.image_left
        reference_surface_areas[cloth_id] = surface_area
    else:
        reference_images[f"Train reference {cloth_id}"] = observation.image_left


In [ ]:
cloth_id = 0
image = reference_images[list(reference_images.keys())[0]]

print(image.shape)



image_cropped = image[20:1020, 660:1660, :] # for references at ICRA
# image_cropped = image[20:1020, 620:1620, :] 

print(image_cropped.shape)

plt.imshow(image_cropped)

In [ ]:
import matplotlib.pyplot as plt

def display_images(images_dict):
    """Displays reference images in a grid layout.

    Args:
        reference_images (dict): A dictionary where keys are cloth IDs and
                                 values are the corresponding images.
    """

    n_images = len(images_dict)
    cols = 4  # Adjust the number of columns as desired
    # cols = 6  # Adjust the number of columns as desired
    rows = (n_images // cols) + (n_images % cols > 0)  # Calculate rows for the grid

    # fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 3, rows * 3.2))  # Create subplots
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(cols * 3, rows * 3.5))  # Create subplots

    cloth_ids = list(images_dict.keys())
    images = list(images_dict.values())

    for i, ax in enumerate(axes.flat):
        if i < n_images:
            image = images[i][20:1020, 660:1660, :]
            # image = images[i][20:1020, 620:1620, :]
            cloth_id = cloth_ids[i]
            ax.imshow(image)
            ax.set_title(f"{cloth_id}")
            ax.axis("off")  # Hide unused subplots
        # else:

    plt.tight_layout()
    plt.show()

# Example usage with your dictionary:
display_images(reference_images) 

In [ ]:
def sample_id_to_reference_id(sample_id):
    return sample_id % 8

def reference_id_to_cloth_type(reference_id):
    if reference_id in [0,1,4,5]:
        return "shirt"
    else:
        return "towel"
    
def reference_id_to_seen(reference_id):
    if reference_id in [0,2,4,6]:
        return True
    else:
        return False

sample_id_to_reference_id(8), sample_id_to_reference_id(10)

In [ ]:
data_dicts = []

team_names = ["AIR_JNU", "Team_Ljubljana", "Ewha_Glab", "SCUT-ROBOT", "test_greater_bay", "SRCB-Test", "Shibata_Lab", "ai_robot", "UOS_robotics", "AIS_team", "3c1s"]

for team_name in team_names:
    team_dir = teams_dir / team_name
    team_sample_dirs = [team_dir / dirname for dirname in sorted(os.listdir(team_dir)) if dirname.startswith("sample_")]

    for i, sample_dir in enumerate(team_sample_dirs):
        
        observation_results_dir = sample_dir / "observation_result"
        
        # Load the surface are
        results_file = observation_results_dir / "result.json"

        with open(results_file, "r") as f:
            result = json.load(f)

        reference_id = sample_id_to_reference_id(i)
        surface_area = result["coverage"]
        reference_surface_area = reference_surface_areas[reference_id]
        coverage = surface_area / reference_surface_area

        data_dict = {
            "team_name": team_name,
            "sample_id": i,
            "reference_id": sample_id_to_reference_id(i),
            "sample_dir": sample_dir,
            "surface_area": surface_area,
            "coverage": coverage,
            "cloth_type": reference_id_to_cloth_type(reference_id),
            "seen": reference_id_to_seen(reference_id),
        }

        data_dicts.append(data_dict)

# convert to pandas dataframe
import pandas as pd

df = pd.DataFrame(data_dicts)

df

In [ ]:
# plot a histogram of the coverages
plt.hist(df["coverage"], bins=10)

In [ ]:
plt.hist(df["coverage"], bins=10)



In [ ]:
import matplotlib.pyplot as plt

# Plot the histogram
plt.figure(figsize=(6, 6))  # Adjust size if needed

# Remove the top and right spines (borders)
ax = plt.gca()  # Get the current Axes instance
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.hist(df["coverage"], bins=20, color="skyblue", edgecolor="white")

# Add titles and labels
plt.title("Distribution of Coverages", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Customize appearance
plt.grid(axis='y', alpha=0.75)  # Subtle grid on y-axis
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Show the plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame for each cloth type
shirts = df[df["cloth_type"] == "shirt"]["coverage"]
towels = df[df["cloth_type"] == "towel"]["coverage"]

# Create the figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()  # Get the current Axes for customization

# Plot the histograms
plt.hist(shirts, bins=20, alpha=0.5, label="Shirts", color="skyblue", edgecolor="white")
plt.hist(towels, bins=20, alpha=0.5, label="Towels", color="lightcoral", edgecolor="white")

# Add titles and labels
plt.title("Coverage Distribution by Cloth Type", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)  # Add a legend to distinguish the histograms

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filter the DataFrame
seen_items = df[df["seen"]]["coverage"]
unseen_items = df[~df["seen"]]["coverage"]

# Create the figure and axes
plt.figure(figsize=(12, 6))
ax = plt.gca()

# Plot the histograms
plt.hist(seen_items, bins=20, alpha=0.5, label="Seen", color="green", edgecolor="white")
plt.hist(unseen_items, bins=20, alpha=0.5, label="Unseen", color="purple", edgecolor="white")

# Add titles and labels
plt.title("Coverage Distribution by Seen Status", fontsize=14)
plt.xlabel("Coverage", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.legend(fontsize=12)

# Customize appearance
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Show the plot
plt.show()


In [ ]:
# print average for shirts, towels, seen, unseen
df.groupby("cloth_type")["coverage"].mean()

In [ ]:
df.groupby("seen")["coverage"].mean()